In [ ]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
IMG_SIZE = 512
MAX_ROOM_SIZE = 5
border_offset = 0.5

In [ ]:
# node = {'jid': '8bba3d84-9d53-4dc6-ad3e-dfac6907b177',
#         'category_id': 'e7a0801e-7abd-4634-b3bc-8777342d124e',
#         'size': {'xLen': 83.64559936523438,
#             'yLen': 83.64070129394531,
#             'zLen': 84.06220245361328},
#         'scale': [1.0, 1.0, 1.0],
#         'pos': [0.2328, 1.6166959838867188, 1.8273],
#         'rot': [0.0, 0.0, 0.0, 1.0],
#         'category': 'Pendant Lamp',
#         'floor': [[-1.548, 3.523],
#             [1.4037, 3.523],
#             [1.4037, -0.1638],
#             [-1.548, -0.1638],
#             [-1.548, 3.523]],
#         'unormalized_bbox': [[-0.1854279968261719,
#             1.6166959838867188,
#             1.4069889877319335],
#             [0.6510279968261719, 2.0348994903564455, 2.247611012268066]],
#         'new_bbox': [[0.46162279471959494, 0.64, 0.42605755336116236],
#             [0.7450038949846435, 0.8139597961425782, 0.6540661311348774]],
#         'type': 'lamp',
#         'rotation': 0.0}

In [ ]:
room_box =  [2.9516999999999998, 2.5, 3.6868000000000003]

In [ ]:
room_min_x = border_offset
room_max_x = room_box[0] + border_offset

room_min_z = border_offset
room_max_z = room_box[2] + border_offset

In [ ]:
node_min_x = (node['new_bbox'][0][0] * room_box[0] + border_offset) / MAX_ROOM_SIZE 
node_max_x = (node['new_bbox'][1][0] * room_box[0] + border_offset) / MAX_ROOM_SIZE 

node_min_z = (node['new_bbox'][0][2] * room_box[2] + border_offset) / MAX_ROOM_SIZE 
node_max_z = (node['new_bbox'][1][2] * room_box[2] + border_offset) / MAX_ROOM_SIZE 

In [ ]:
node['new_bbox'][0][0] * room_box[0]

In [ ]:
node['new_bbox'][1][2] * room_box[2]

In [ ]:
node_max_x

In [ ]:
node_min_x = int(node_min_x * IMG_SIZE)
node_max_x = int(node_max_x *IMG_SIZE)

node_min_z = int(node_min_z * IMG_SIZE)
node_max_z = int(node_max_z *IMG_SIZE)

In [ ]:
node_img = np.zeros((512,512))
node_img = cv2.rectangle(node_img,(node_min_x,node_min_z),(node_max_x,node_max_z),(255,255,255),-1)

In [ ]:
np.max(node_img)

In [ ]:
plt.imshow(node_img, cmap="gray")

In [ ]:
node_floor = np.array(node['floor'])

In [ ]:
node_floor_min_x = np.min(node_floor[:,0])
node_floor[:,0] -= node_floor_min_x - border_offset

node_floor_min_z = np.min(node_floor[:,1])
node_floor[:,1] -= node_floor_min_z - border_offset


In [ ]:
points = ((node_floor / MAX_ROOM_SIZE)*IMG_SIZE).astype(int)

In [ ]:
wall_img = np.zeros((512,512))
wall_img = cv2.polylines(wall_img, [points], True, (255,255,255), 10)

In [ ]:
plt.imshow(wall_img, cmap = "gray")

In [ ]:
floor_img = np.zeros((512,512))
floor_img = cv2.fillPoly(floor_img, pts =[points], color=(255,255,255))

In [ ]:
plt.imshow(floor_img, cmap="gray")

In [ ]:
from new.new_rendered import *
import matplotlib.pyplot as plt

In [ ]:
all_room_info = json.load(open("/home/ubuntu/research/3D-FRONT-ToolBox/metadata43DSLN/all_room_info_b.json"))

In [ ]:
room_info_dict = all_room_info['5767']

In [ ]:
scene = NewRenderedScene(room_info_dict)

In [ ]:
plt.imshow(scene.wall,cmap="gray")

In [ ]:
composite = scene.create_composite()

In [ ]:
room_info_dict['valid_objects']

In [ ]:
node = scene.object_nodes[4]

In [ ]:
composite.add_node(node)

In [ ]:
composite.cat_map.shape

In [ ]:
NewRenderedScene.cat_to_index[node["type"]]

In [ ]:
a = composite.height_map.data.numpy()

In [ ]:
plt.imshow(a,cmap="gray")

In [ ]:
batch_data = composite.get_composite()

In [ ]:
batch_data.shape

In [ ]:
for key,rpom_dict in all_room_info.items():
    print(rpom_dict['bbox'])

# -------------------------------

In [ ]:
data_dir = "/home/ubuntu/research/3D-FRONT-ToolBox/metadata43DSLN/all_room_info_b.json"

In [ ]:
from new.new_continue_dataset import *

In [ ]:
train_dataset = NewShouldContinueDataset(data_dir)

In [ ]:
import numpy as np

In [ ]:
train_dataset.all_room_keys[452]

--------------------

In [ ]:
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import json

In [ ]:
from new.new_continue_dataset import NewShouldContinueDataset
from new.new_rendered import NewRenderedScene

In [ ]:
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import math
import models
from models import *
import os
import os.path
import numpy as np
import utils

In [ ]:
"""
Train the network that predicts whether we should continue adding objects
"""
parser = argparse.ArgumentParser(description='Should Continue')
parser.add_argument('--data-dir', type=str, default="bedroom", metavar='S')
parser.add_argument('--num-workers', type=int, default=6, metavar='N')
parser.add_argument('--last-epoch', type=int, default=-1, metavar='N')
parser.add_argument('--train-size', type=int, default=6400, metavar='N')
parser.add_argument('--save-dir', type=str, default="train/bedroom", metavar='S')
parser.add_argument('--ablation', type=str, default=None, metavar='S')
parser.add_argument('--lr', type=float, default=0.001, metavar='N')
parser.add_argument('--eps', type=float, default=1e-6, metavar='N')
parser.add_argument('--use-count', action='store_true', default=False)

In [ ]:
args = parser.parse_args("")

In [ ]:
save_dir = args.save_dir
utils.ensuredir(save_dir)
learning_rate = args.lr
batch_size = 16

In [ ]:
num_categories = len(json.load(open("new/valid_types.json")))

In [ ]:
num_categories

In [ ]:
if args.ablation is None:
    num_input_channels = num_categories+8
elif args.ablation == "basic":
    num_input_channels = 6
elif args.ablation == "depth":
    num_input_channels = 1
else:
    raise NotImplementedError

In [ ]:
logfile = open(f"{save_dir}/log_continue.txt", 'w')
def LOG(msg):
    print(msg)
    logfile.write(msg + '\n')
    logfile.flush()

In [ ]:
if args.ablation is None:
    num_input_channels = num_categories+8
elif args.ablation == "basic":
    num_input_channels = 6
elif args.ablation == "depth":
    num_input_channels = 1
else:
    raise NotImplementedErrorLOG('Building model...')
if args.use_count:
    model = models.resnet101(num_classes=2, num_input_channels=num_input_channels, use_fc=False)
    fc = FullyConnected(2048+num_categories, 2)
else:
    model = models.resnet101(num_classes=2, num_input_channels=num_categories+8)

In [ ]:
loss = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=1)

In [ ]:
LOG('Converting to CUDA...')
model.cuda()
if args.use_count:
    fc.cuda()
loss.cuda()
softmax.cuda()

In [ ]:
args.data_dir = "/home/ubuntu/research/3D-FRONT-ToolBox/metadata43DSLN/all_room_info_b.json"

In [ ]:
LOG('Building dataset...')
train_dataset = NewShouldContinueDataset(
    data_dir = args.data_dir,
    complete_prob = 0.5,
    ablation = args.ablation
)

In [ ]:
len(train_dataset)

In [ ]:
# for i in range(16+8):
#     print(i)
#     plt.imshow(train_dataset[3][0][i].data.numpy(), cmap="gray")
#     plt.show()

In [ ]:
validation_dataset = NewShouldContinueDataset(
    data_dir = args.data_dir, #TODO change to valid dir
    complete_prob = 0.5,
    ablation = args.ablation
)

In [ ]:
LOG('Building data loader...')
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    num_workers = args.num_workers,
    shuffle = True
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size = batch_size,
    num_workers = 0,
    shuffle = True
)

In [ ]:
LOG('Building optimizer...')
optimizer = optim.Adam(list(model.parameters()), # +list(fc.parameters()
    lr = learning_rate,
    betas = (0.9,0.999),
    eps = args.eps
)

In [ ]:
if args.last_epoch < 0:
    load = False
    starting_epoch = 0
else:
    load = True
    last_epoch = args.last_epoch

In [ ]:
if load:
    LOG('Loading saved models...')
    model.load_state_dict(torch.load(f"{save_dir}/continue_{last_epoch}.pt"))
    if args.use_count:
        fc.load_state_dict(torch.load(f"{save_dir}/continue_fc_{last_epoch}.pt"))
    optimizer.load_state_dict(torch.load(f"{save_dir}/continue_optim_backup.pt"))
    starting_epoch = last_epoch + 1

In [ ]:
current_epoch = starting_epoch
num_seen = 0

model.train()

# Train

In [ ]:
def validate():
    model.eval()
    #fc.eval()
    total_loss = []
    total_accuracy = []
    for _, (data, target, existing) in enumerate(validation_loader):
        with torch.no_grad():
            data, target = data.cuda(), target.cuda()
            if args.use_count:
                existing = existing.cuda()
                existing = Variable(existing)

            optimizer.zero_grad()

            if args.use_count:
                o_conv = model(data)
                o_conv = torch.cat([o_conv, existing], 1)
                output = fc(o_conv)
            else:
                output = model(data)

            loss_val = loss(output, target.long())
            total_loss.append(loss_val.item())

            output = softmax(output)
            outputs = output.cpu().data.numpy()
            targets = target.cpu().data.numpy()
            predictions = np.argmax(outputs, axis=1)
            num_correct = np.mean(predictions == targets)
            total_accuracy.append(num_correct)

    LOG(f'Loss: {np.mean(total_loss)}, Accuracy: {np.mean(total_accuracy)}')

In [ ]:
total_epoch = 200

In [ ]:
for ep in range(total_epoch):
    print("Epoch:", ep)
    train_loss_list = []
    for batch_idx, (data, target, existing) in tqdm(enumerate(train_loader)):
            #print(batch_idx)
            data, target = data.cuda(), target.cuda()

            if args.use_count:
                existing = existing.cuda()
                existing = Variable(existing)

            optimizer.zero_grad()
            if args.use_count:
                o_conv = model(data)
                o_conv = torch.cat([o_conv, existing], 1)
                output = fc(o_conv)
            else:
                output = model(data)

            loss_val = loss(output, target.long())
            loss_val.backward()
            optimizer.step()
            
            train_loss_list.append(loss_val.item())

            num_seen += batch_size
            if num_seen % 800 == 0:
                LOG(f'Examples {num_seen}/10000 Train Loss {np.mean(train_loss_list)}')
                train_loss_list.clear()
            if num_seen % 10000 == 0:
                LOG('Validating')
                validate()
                model.train()
                #fc.train()
                num_seen = 0
                current_epoch += 1
                LOG(f'=========================== Epoch {current_epoch} ===========================')
                if current_epoch % 2 == 0:
                    torch.save(model.state_dict(), f"{save_dir}/continue_{current_epoch}.pt")
                    if args.use_count:
                        torch.save(fc.state_dict(), f"{save_dir}/continue_fc_{current_epoch}.pt")
                    torch.save(optimizer.state_dict(), f"{save_dir}/continue_optim_backup.pt")
